In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import time
import sys
import os
import csv
from dotenv import load_dotenv
load_dotenv()

In [ ]:
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled')
service = Service(
    r"/path/to/geckodriver"
)
driver = webdriver.Firefox(service=service, options=options)

In [ ]:
actions = ActionChains(driver)

In [ ]:
driver.get("https://www.instagram.com/")
sleep(3)

In [ ]:
# enter username
username = driver.find_element(By.NAME, "username")
username.send_keys(os.getenv("USERNAME"))
sleep(2)

# enter password
password = driver.find_element(By.NAME, "password")
password.send_keys(os.getenv("PASSWORD"))
sleep(1)
password.submit()
sleep(5)

In [ ]:
csv_file = 'instagram_comments.csv'
csv_columns = ['username', 'comment', 'likes', 'datetime']
with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(csv_columns)

In [ ]:
post_url = "https://www.instagram.com/p/C9iUb6wJyPy/" # 362 comments
driver.get(post_url)
sleep(3)

In [ ]:
# extract things from the big black box of comments
comments_div = driver.find_elements(By.XPATH, "//div[contains(@class, 'x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1iyjqo2 x2lwn1j xeuugli x1q0g3np xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1')]")

In [ ]:
len(comments_div)

In [ ]:
actions.move_to_element(comments_div[0]).perform()
sleep(1)
actions.move_by_offset(120, 0).perform()
sleep(2)
actions.move_by_offset(0, -55).perform()
sleep(1)
actions.click().perform()
sleep(1)

In [ ]:
max_constant_iterations = 2
constant_iterations = 0
previous_length = 0
processed_usernames = set()

In [ ]:
while constant_iterations < max_constant_iterations:
    # Scroll and update comments_div
    start_time = time.time()
    while time.time() - start_time < 5:  # Scroll for 5 seconds
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(0.2)

    comments_div = driver.find_elements(By.XPATH, "//div[contains(@class, 'x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1iyjqo2 x2lwn1j xeuugli x1q0g3np xqjyukv x1qjc9v5 x1oa3qoh x1nhvcw1')]")
    current_length = len(comments_div)
    print(f"----{current_length}---------")

    if current_length == previous_length:
        print("----INSIDE CONSTANT ITERATION----")
        constant_iterations += 1
    else:
        constant_iterations = 0

    previous_length = current_length

    # Scrape comments
    for com in comments_div:
        try:
            # Extract username
            try:
                username_span = com.find_element(By.XPATH, ".//span[contains(@class, '_ap3a _aaco')]")
                username = username_span.text
            except Exception as e:
                username = ""
                print(f"Error extracting username: {str(e)}")

            if username not in processed_usernames:
                print(f"---------Comment by {username}-------------")
                processed_usernames.add(username)

                # Extract comment text
                try:
                    comment_div = com.find_element(By.XPATH, ".//div[contains(@class, 'x9f619 xjbqb8w x78zum5 x168nmei x13lgxp2 x5pf9jr xo71vjh x1uhb9sk x1plvlek xryxfnj x1c4vz4f x2lah0s xdt5ytf xqjyukv x1cy8zhl x1oa3qoh x1nhvcw1')]")
                    comment_span = comment_div.find_element(By.XPATH, ".//span[contains(@class, 'x1lliihq x1plvlek xryxfnj x1n2onr6 x193iq5w xeuugli x1fj9vlw x13faqbe x1vvkbs x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x1i0vuye xvs91rp xo1l8bm x5n08af x10wh9bi x1wdrske x8viiok x18hxmgj')]")
                    comment_text = comment_span.text
                except Exception as e:
                    comment_text = ""
                    print(f"Error extracting comment text: {str(e)}")

                # print(f"TEXT - {comment_text}")

                # Extract number of likes
                try:
                    likes_div = com.find_element(By.XPATH, ".//div[contains(@class, 'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz xq8finb x87ps6o x1d5wrs8')]")
                    likes_span = likes_div.find_element(By.XPATH, ".//span[contains(@class, 'x1lliihq x193iq5w x6ikm8r x10wlt62 xlyipyv xuxw1ft')]")
                    likes = likes_span.text
                except Exception as e:
                    likes = ""
                    print(f"Error extracting likes: {str(e)}")

               # print(f"LIKES - {likes}")

                # Extract datetime
                try:
                    time_html = com.find_element(By.XPATH, ".//time[contains(@class, 'x1ejq31n xd10rxx x1sy0etr x17r0tee x1roi4f4 xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6')]")
                    datetime_value = time_html.get_attribute('datetime')
                    # print(f"TIME - {datetime_value} - {time_html.text}")
                except Exception as e:
                    datetime_value = ""
                    print(f"Error extracting datetime: {str(e)}")

                # Append data to CSV
                with open(csv_file, 'a', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([username, comment_text, likes, datetime_value])

        except Exception as e:
            print(f"Error while scraping comment: {str(e)}")
            pass

In [ ]:
print(f"Total comments loaded: {len(comments_div)}")

In [ ]:
# ABOVE CODE GIVES AN ERROR WHEN COMMENT IS GIF - hence no text is found

In [ ]:
driver.quit()